<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-09 10:42:51
-------------------
qualified stocks: 88
with latest results: 29
still star stocks: 16
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.11 L
Current:  1.35 C
-------------------
Today PnL: 1.42 L (1.06%)
Current PnL: -14.90 L (-10.38%)
CY Booked + Current PnL: -8.82 L (-6.14%)
-------------------
Total profit:  3.92 L
Total loss:  -18.82 L
-------------------
Total Booked + Current PnL: 18.16 L (15.58%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.51%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 69.83 L (51.8%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LICI,785.05,983.0,-2.96,H-LC,85.23,238650.0,43958.0,5131.0,0.26,22.58,2.15,25.21,31.0,8.57,1.90,31.97,XY25,NTT,INSURANCE
80,UNIONBANK,123.87,163.0,1.23,M-LC,93.18,178259.0,37419.0,7077.0,2.28,26.57,3.97,31.59,81.0,5.29,1.42,54.84,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.95,104406.0,26044.0,7987.0,-0.08,33.24,7.65,43.43,71.0,3.26,0.83,39.64,XY24,NTT,MISC
57,RELIANCE,1291.83,1526.0,6.12,X-LC,35.23,157908.0,17099.0,8432.0,0.36,12.14,5.34,18.13,20.0,2.03,1.26,24.68,XY25,NTT,REFINERIES
73,TATAELXSI,7332.28,7332.0,-12.79,X-MC,64.77,91560.0,-11092.0,11088.0,1.04,-10.81,12.11,-0.00,51.0,-1.00,0.73,37.62,OX40N,NTT,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,55.68,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,246.0,-0.56,0.51,55.40,XR,NTT,HOTELS
11,BANDHANBNK,214.25,400.00,-28.43,H-SC,78.41,213927.0,-34603.0,250081.0,6.26,-13.92,116.90,86.70,118.0,-0.14,1.70,41.52,XY24,NTT,BANKS
29,HAPPSTMNDS,919.80,1491.84,-22.57,M-SC,47.73,65315.0,-31264.0,91330.0,5.63,-32.37,139.83,62.19,195.0,-0.34,0.52,14.53,AR,ATH,IT
55,RBLBANK,230.61,388.00,49.48,M-SC,72.73,232420.0,-1880.0,161788.0,5.11,-0.80,69.61,68.25,229.0,-0.01,1.85,53.06,XY24,NTT,BANKS
0,5PAISA,564.75,565.00,163.22,H-SC,84.09,141949.0,-44983.0,45069.0,4.69,-24.06,31.75,0.04,138.0,-1.00,1.13,35.80,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,558.54,680.00,-15.50,H-MC,40.91,181435.0,4378.0,34128.0,-1.57,2.47,18.81,21.75,106.0,0.13,1.44,29.78,XY24,NTT,PAINTS
37,INDIAMART,2327.09,4871.06,-51.09,H-SC,45.45,127518.0,4182.0,130642.0,-0.91,3.39,102.45,109.32,111.0,0.03,1.01,25.98,AR,ATH,MISC
67,SIS,477.00,477.00,2205.36,M-SC,59.09,62407.0,-18683.0,18685.0,-0.77,-23.04,29.94,0.00,211.0,-1.00,0.50,26.48,OX40N,NTT,MISC
17,CAMPUS,294.86,393.00,NaN,NaN,88.64,168794.0,1019.0,54824.0,-0.74,0.61,32.48,33.28,188.0,0.02,1.34,30.68,XY24,NTT,FOOTWEAR
6,APOLLOHOSP,6904.43,8285.00,-16.22,H-LC,22.73,103500.0,-66.0,20772.0,-0.61,-0.06,20.07,20.00,33.0,-0.00,0.82,14.26,X40N,BTT,HEALTHCARE


In [12]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,52.77,53.0,48.19,M-SC,75.00,128790.0,-13689.0,14309.0,-0.27,-9.61,11.11,0.44,227.0,-0.96,1.02,52.01,OX40N,NTT,BANKS
71,SYMPHONY,1306.42,1306.0,-28.23,M-SC,1.14,153047.0,-18094.0,18044.0,1.54,-10.57,11.79,-0.03,178.0,-1.00,1.22,10.00,OX40N,NTT,DURABLES
73,TATAELXSI,7332.28,7332.0,-12.79,X-MC,64.77,91560.0,-11092.0,11088.0,1.04,-10.81,12.11,-0.00,51.0,-1.00,0.73,37.62,OX40N,NTT,IT
45,KANSAINER,299.63,340.0,-67.67,H-SC,6.82,225000.0,-44667.0,81000.0,0.34,-16.56,36.00,13.47,134.0,-0.55,1.79,12.62,XY24,NTT,PAINTS
78,TTKPRESTIG,769.29,770.0,-10.66,X-SC,17.05,81620.0,-19157.0,19254.0,0.27,-19.01,23.59,0.09,64.0,-0.99,0.65,6.21,OX40N,NTT,DURABLES


In [13]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,3794.03,4998.00,-18.09,X-LC,11.36,266526.0,-29408.0,123322.0,0.97,-9.94,46.27,31.73,1.0,-0.24,2.12,6.68,X40,BTT,IT
40,INFY,1461.46,2275.00,-12.06,X-LC,39.77,282283.0,20682.0,124938.0,0.84,7.91,44.26,55.67,2.0,0.17,2.25,14.49,X40,BTT,IT
49,LTIM,5564.16,7197.33,-1.63,X-LC,65.91,189360.0,-10950.0,69741.0,1.08,-5.47,36.83,29.35,3.0,-0.16,1.51,30.82,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,7.95,152878.0,-48508.0,136123.0,0.13,-24.09,89.04,43.51,5.0,-0.36,1.22,5.45,X40,BTT,PAINTS
33,HINDUNILVR,2413.81,2723.00,-14.85,X-LC,20.45,267579.0,-2768.0,37408.0,-0.03,-1.02,13.98,12.81,6.0,-0.07,2.13,10.74,XY25,NTT,FMCG


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UNIONBANK,123.87,163.0,1.23,M-LC,93.18,178259.0,37419.0,7077.0,2.28,26.57,3.97,31.59,81.0,5.29,1.42,54.84,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.95,104406.0,26044.0,7987.0,-0.08,33.24,7.65,43.43,71.0,3.26,0.83,39.64,XY24,NTT,MISC
48,LICI,785.05,983.0,-2.96,H-LC,85.23,238650.0,43958.0,5131.0,0.26,22.58,2.15,25.21,31.0,8.57,1.90,31.97,XY25,NTT,INSURANCE


In [15]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,130.64,228.00,104.28,M-MC,89.77,232177.0,7215.0,160434.0,-0.16,3.21,69.10,74.53,176.0,0.04,1.85,34.80,XY24,BTT,STEEL
18,CAMS,3643.00,5250.99,6.44,H-SC,87.50,119664.0,17660.0,27367.0,0.27,17.31,22.87,44.14,115.0,0.65,0.95,36.37,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,3.96,M-LC,57.95,104406.0,26044.0,7987.0,-0.08,33.24,7.65,43.43,71.0,3.26,0.83,39.64,XY24,NTT,MISC
80,UNIONBANK,123.87,163.00,1.23,M-LC,93.18,178259.0,37419.0,7077.0,2.28,26.57,3.97,31.59,81.0,5.29,1.42,54.84,XY24,NTT,BANKS
87,WIPRO,243.46,420.00,-9.26,M-LC,26.14,155310.0,4365.0,105083.0,0.77,2.89,67.66,72.51,68.0,0.04,1.24,7.51,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,113.49,190.00,-24.57,M-MC,76.14,146950.0,17798.0,69272.0,3.82,13.78,47.14,67.42,158.0,0.26,1.17,42.92,XR,NTT,BANKS
86,WHIRLPOOL,1167.49,2270.00,-32.01,M-SC,52.27,198127.0,26506.0,135558.0,4.61,15.44,68.42,94.43,183.0,0.20,1.58,46.23,XR,NTT,DURABLES
87,WIPRO,243.46,420.00,-9.26,M-LC,26.14,155310.0,4365.0,105083.0,0.77,2.89,67.66,72.51,68.0,0.04,1.24,7.51,XR,NTT,IT
37,INDIAMART,2327.09,4871.06,-51.09,H-SC,45.45,127518.0,4182.0,130642.0,-0.91,3.39,102.45,109.32,111.0,0.03,1.01,25.98,AR,ATH,MISC
1,AARTIIND,487.04,919.00,40.73,M-SC,42.05,80878.0,-458.0,72596.0,1.56,-0.56,89.76,88.69,213.0,-0.01,0.64,36.38,XR,NTT,CHEMICALS


In [17]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.0,-57.75,L-SC,55.68,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,246.0,-0.56,0.51,55.40,XR,NTT,HOTELS
50,MASFIN,326.60,399.5,-20.51,H-SC,73.86,90540.0,-7440.0,29308.0,2.39,-7.59,32.37,22.32,127.0,-0.25,0.72,31.31,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.95,104406.0,26044.0,7987.0,-0.08,33.24,7.65,43.43,71.0,3.26,0.83,39.64,XY24,NTT,MISC
82,VALIANTORG,512.64,838.0,-363.88,H-SC,100.00,112060.0,-21226.0,105818.0,-0.16,-15.93,94.43,63.47,135.0,-0.20,0.89,87.84,XR,NTT,CHEMICALS
70,SURYODAY,189.78,240.0,64.47,H-SC,94.32,117023.0,-32903.0,72578.0,2.19,-21.95,62.02,26.46,132.0,-0.45,0.93,49.91,XR,NTT,BANKS


In [18]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,512.64,838.00,-363.88,H-SC,100.00,112060.0,-21226.0,105818.0,-0.16,-15.93,94.43,63.47,135.0,-0.20,0.89,87.84,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,6561.11,M-SC,98.86,73085.0,-20725.0,97342.0,-0.44,-22.09,133.19,81.67,222.0,-0.21,0.58,41.53,XR,NTT,CERAMICS
76,TITAGARH,1097.23,1548.00,-6.04,H-SC,97.73,165174.0,-26841.0,105728.0,0.31,-13.98,64.01,41.08,131.0,-0.25,1.31,39.80,XY24,NTT,ENGINEERING
72,TANLA,917.30,1963.11,-32.99,H-SC,96.59,163804.0,-75611.0,348575.0,0.76,-31.58,212.80,114.01,117.0,-0.22,1.30,50.23,AR,ATH,IT
69,STARHEALTH,551.56,761.00,4.42,M-SC,95.45,170610.0,-29606.0,105625.0,-0.11,-14.79,61.91,37.97,202.0,-0.28,1.36,36.99,XY24,NTT,INSURANCE


In [19]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.42
1,25,43.83
2,50,73.43


In [20]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.68
XY25     15.38
XR       11.60
X40      10.63
X40N      8.90
OX40N     8.74
AR        8.35
X200      1.51
SR        1.23
Name: CurrAlloc%, dtype: float64

In [21]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.60
X-LC    19.03
M-SC    17.49
H-MC     8.20
M-LC     8.18
H-LC     7.47
M-MC     5.82
X-MC     5.76
X-SC     2.90
L-SC     2.49
L-LC     1.19
Name: CurrAlloc%, dtype: float64

In [22]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.09,-1.22,32.92
IT,10.81,-15.00,75.74
BANKS,10.58,-5.36,52.37
MISC,6.82,-26.29,75.67
FINANCE,6.41,-18.56,64.80
INSURANCE,5.58,4.93,23.47
PAINTS,5.51,-18.69,42.13
ELECTRICAL,5.44,-2.89,47.51
HEALTHCARE,4.54,-3.77,34.74


In [23]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2740295.0,26
XR,1182106.0,14
AR,1021143.0,9
X40,593557.0,7
XY25,445447.0,10
X40N,409829.0,8
OX40N,331661.0,11
SR,189663.0,2
X200,69741.0,1


In [24]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1899570.0,18
M-SC,1677551.0,20
X-LC,823034.0,11
M-MC,536869.0,5
H-MC,467354.0,7
L-SC,285538.0,4
X-MC,284965.0,5
M-LC,278784.0,6
X-SC,267214.0,4


In [25]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,882528.0,6
M-SC,XY24,728668.0,6
H-SC,AR,537937.0,3
X-LC,X40,466967.0,4
M-SC,XR,421739.0,5
M-MC,XY24,381848.0,3
H-SC,XR,207704.0,3
H-MC,XY24,172310.0,2
M-SC,OX40N,165231.0,5


In [26]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
